### Regresión Lineal Multivariable
---
#### Escala de características

Este es un paso esencial en la fase de pre-procesamiento ya que la mayoría de los algoritmos de Machine Learning tienen mucho mejor rendimiento cuando tratan con características que están en la misma escala. Las técnicas más comunes son:

* <b>Normalización:</b> se refiere a reescalar las características en un rango $[0, 1]$, que es un caso especial de escalación "min-max". Para normalizar nuestros datos, simplemente necesitaremos aplicar el método de escalación "min-max" a cada columna de características.

$$ X_{changed} = {{X-X_{min}}\over{X_{max} - X_{min}}} $$

* <b>Estandarización</b>: consiste en centrar las columnas de características con respecto a media $0$ con desviación estándar $1$ de forma que las columnas de características tengan los mismos parámetros de una distribución normal estándar (media cero y varianza uno). De esta forma es mucho más fácil para los algoritmos de aprendizaje "aprender" los pesos de los parámetros. Adicionalmente mantiene información útil sobre valores atípicos y hace los algoritmos menos sencibles a ellos.
    
    ![](images/Estandarizacion.png)

### Programming

In [1]:
from pylab import *

In [2]:
from sklearn.datasets import load_boston

In [3]:
boston_dataset = load_boston()

In [4]:
import pandas as pd

In [5]:
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston['MEDV'] = boston_dataset.target
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
import torch

#### Normalización

In [7]:
X = array([boston['RM'], boston['AGE']])
Y = array(boston['MEDV'])

In [8]:
X = torch.Tensor(X.T)
Y = torch.Tensor(Y.reshape(506, 1))

In [9]:
X_min = torch.min(X, dim=0)

print(X_min)

torch.return_types.min(
values=tensor([3.5610, 2.9000]),
indices=tensor([365,  41]))


In [10]:
X_max = torch.max(X, dim=0)

print(X_max)

torch.return_types.max(
values=tensor([  8.7800, 100.0000]),
indices=tensor([364,   8]))


In [11]:
X = (X - X_min.values) / (X_max.values - X_min.values)

In [12]:
### Train
!rm -r runs

In [13]:
class LinearModel(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        
        self.F = torch.nn.Linear(input_dim, 1)
        
        self.loss = None
        
    def forward(self, x):
        return self.F(x)

In [14]:
# Para debuguear
from torch.utils.tensorboard import SummaryWriter

In [15]:
# Función de entrenamiento
def train(model, x, y, epochs=1, lr=0.01):
    tb = SummaryWriter()
    
    # Función de costo (MEAN SQUARED ERROR)
    loss_fn = torch.nn.MSELoss()
    # Optimizador SGD (Gradient Descent)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    model.train()
    
    for epoch in range(epochs):
        
        preds = model(x)
        loss = loss_fn(preds, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        model.loss = loss.item()

        tb.add_scalar('Loss', loss.item(), epoch)
        
    tb.close()

In [16]:
model = LinearModel(2)

In [17]:
train(model, X, Y, epochs=500, lr=0.5)

In [18]:
print(model.loss)

39.65400695800781


![](images/Loss_1.svg)

Se observa una mejora significativa con respecto a las implementaciones anteriores donde al elevar el valor del <b>learning rate</b> el algoritmo de aprendizaje no era capaz de converger en un valor óptimo. En este caso se ha elevado significativamente el valor del <b>learning rate</b> permitiendo reducir la cantidad de ciclos de entrenamiento necesarios para optener los parámetros correctos.

#### Estandarización

In [19]:
X = array([boston['RM'], boston['AGE']])
Y = array(boston['MEDV'])

In [20]:
X = torch.Tensor(X.T)
Y = torch.Tensor(Y.reshape(506, 1))

In [21]:
X_mean = torch.mean(X, dim=0) # Media
X_var = torch.var(X, dim=0)   # Varianza

X = (X - X_mean) / torch.sqrt(X_var)

In [22]:
model = LinearModel(2)

In [23]:
!rm -r runs

In [24]:
train(model, X, Y, epochs=20, lr=0.5)

In [25]:
print(model.loss)

39.6539306640625


En este caso se obtiene un resultado muchísimo más satisfactorio logrando obtener los parámetros correctos del modelo con una cantidad de ciclos significativamente menor al anterior.

![](images/Loss_2.svg)